In [2]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
%autoreload 2

In [4]:
#! /usr/bin/python3
import argparse
import logging
import os
import sys
from collections import namedtuple

import torch
from torch.utils.data import DataLoader, SequentialSampler
from tqdm import tqdm

#
from transformers import BertTokenizer
#from transformers import BertAbs



I0108 21:29:54.515745 140163355588416 file_utils.py:35] PyTorch version 1.3.0 available.


In [5]:
sys.path.insert(0, "/dadendev/transformers/examples/summarization")
from modeling_bertabs import BertAbs, build_predictor

from utils_summarization import (
    #SummarizationDataset,
    build_mask,
    compute_token_type_ids,
    encode_for_summarization,
    fit_to_block_size,
)
from run_summarization import format_summary

In [6]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)
model = BertAbs.from_pretrained("bertabs-finetuned-cnndm")
model.to("cuda")
model.eval()

symbols = {
    "BOS": tokenizer.vocab["[unused0]"],
    "EOS": tokenizer.vocab["[unused1]"],
    "PAD": tokenizer.vocab["[PAD]"],
}

I0108 21:27:26.882526 139788440540992 tokenization_utils.py:398] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/daden/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


AttributeError: type object 'BertAbs' has no attribute 'from_pretrained'

In [6]:
nlp_path = os.path.abspath("../../")
if nlp_path not in sys.path:
    sys.path.insert(0, nlp_path)
from utils_nlp.models.transformers.extractive_summarization import Bunch
args = Bunch({"block_trigram":True, "alpha": 0.95, "beam_size": 5, "min_length": 20, "max_length": 200})

In [8]:
predictor = build_predictor(args, tokenizer, symbols, model)

NameError: name 'symbols' is not defined

In [7]:
import os
import shutil
import sys
from tempfile import TemporaryDirectory
import torch

nlp_path = os.path.abspath("../../")
if nlp_path not in sys.path:
    sys.path.insert(0, nlp_path)

from utils_nlp.common.pytorch_utils import get_device
from utils_nlp.dataset.cnndm import CNNDMBertSumProcessedData, CNNDMSummarizationDataset
from utils_nlp.eval.evaluate_summarization import get_rouge
from utils_nlp.models.transformers.extractive_summarization import (
    ExtractiveSummarizer,
    ExtSumProcessedData,
    ExtSumProcessor,
)

import numpy as np
import pandas as pd
import scrapbook as sb

[nltk_data] Downloading package punkt to /home/daden/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
#from utils_nlp.models.transformers.datasets import SummarizationDataset
from utils_nlp.dataset.cnndm import CNNDMAbsSumDataset, CNNDMSummarizationDataset
#def build_data_iterator(args, tokenizer):


In [9]:
DATA_PATH = '/tmp/tmpsh6mbj3g'

In [10]:
QUICK_RUN = True
# the data path used to save the downloaded data file
# DATA_PATH = TemporaryDirectory().name
# The number of lines at the head of data file used for preprocessing. -1 means all the lines.
TOP_N = 4
CHUNK_SIZE=200
if not QUICK_RUN:
    TOP_N = -1
    CHUNK_SIZE = 2000

In [11]:
from torch.utils.data import Dataset
class SummarizationDataset(Dataset):
    def __init__(self, source, target=None):
        self.source = source
        self.target = target
    def __len__(self):
        return len(self.source)
    def __getitem__(self, idx):
        return self.source[idx], self.target[idx]

In [12]:
train_dataset, test_dataset = CNNDMAbsSumDataset(top_n=TOP_N, local_cache_path=DATA_PATH)

I0108 21:30:09.018494 140163355588416 utils.py:173] Opening tar file /tmp/tmpsh6mbj3g/cnndm.tar.gz.
I0108 21:30:09.020582 140163355588416 utils.py:181] /tmp/tmpsh6mbj3g/test.txt.src already extracted.
I0108 21:30:09.306402 140163355588416 utils.py:181] /tmp/tmpsh6mbj3g/test.txt.tgt.tagged already extracted.
I0108 21:30:09.332876 140163355588416 utils.py:181] /tmp/tmpsh6mbj3g/train.txt.src already extracted.
I0108 21:30:16.825703 140163355588416 utils.py:181] /tmp/tmpsh6mbj3g/train.txt.tgt.tagged already extracted.
I0108 21:30:17.439162 140163355588416 utils.py:181] /tmp/tmpsh6mbj3g/val.txt.src already extracted.
I0108 21:30:17.773054 140163355588416 utils.py:181] /tmp/tmpsh6mbj3g/val.txt.tgt.tagged already extracted.


In [13]:
data = list(test_dataset.get_source()), list(test_dataset.get_target())
test_sum_dataset = SummarizationDataset(data[0], data[1])

In [16]:
len(data[1])

4

In [27]:
data = list(train_dataset.get_source()), list(train_dataset.get_target())
train_sum_dataset = SummarizationDataset(data[0], data[1])

In [15]:
Batch = namedtuple("Batch", [ "batch_size", "src", "segs", "mask_src", "tgt", "tgt_str"])

In [23]:
def collate(data, tokenizer, block_size, device):
    """ Collate formats the data passed to the data loader.
    In particular we tokenize the data batch after batch to avoid keeping them
    all in memory. We output the data as a namedtuple to fit the original BertAbs's
    API.
    """
    data = [x for x in data if not len(x[1]) == 0]  # remove empty_files
    #print(data)
    #names = [name for name, _, _ in data]
    # summaries = [" ".join(summary_list) for _, _, summary_list in data]
    summaries = [" ".join(summary_list) for _, summary_list in data]
  

    encoded_text = [encode_for_summarization(story, summary, tokenizer) for story, summary in data]
    
    
    #""""""
    encoded_stories = torch.tensor(
        [fit_to_block_size(story, block_size, tokenizer.pad_token_id) for story, _ in encoded_text]
    )
    encoder_token_type_ids = compute_token_type_ids(encoded_stories, tokenizer.cls_token_id)
    encoder_mask = build_mask(encoded_stories, tokenizer.pad_token_id)
    #"""
    encoded_summaries= torch.tensor(
        [fit_to_block_size(summary, block_size, tokenizer.pad_token_id) for _, summary in encoded_text]
    )
    
    print(len(encoded_stories))

    batch = Batch(
        #document_names=None,
        batch_size=len(encoded_stories),
        src=encoded_stories.to(device),
        segs=encoder_token_type_ids.to(device),
        mask_src=encoder_mask.to(device),
        tgt=encoded_summaries.to(device),
        tgt_str=summaries,
    )
    return batch

In [24]:
def encode_for_summarization(story_lines, summary_lines, tokenizer, max_len=512):
    """ Encode the story and summary lines, and join them
    as specified in [1] by using `[SEP] [CLS]` tokens to separate
    sentences.
    """
    story_lines_token_ids = [tokenizer.encode(line, max_length=max_len) for line in story_lines]
    story_token_ids = [token for sentence in story_lines_token_ids for token in sentence]
    summary_lines_token_ids = [tokenizer.encode(line, max_length=max_len) for line in summary_lines]
    summary_token_ids = [token for sentence in summary_lines_token_ids for token in sentence]

    return story_token_ids, summary_token_ids

In [25]:
def build_data_iterator(dataset, tokenizer, batch_size=16, device='cuda'):
   
    sampler = SequentialSampler(dataset)

    def collate_fn(data):
        return collate(data, tokenizer, block_size=512, device=device)

    iterator = DataLoader(dataset, sampler=sampler, batch_size=batch_size, collate_fn=collate_fn,)

    return iterator

In [26]:
from utils_nlp.common.pytorch_utils import get_device
device, num_gpus = get_device(num_gpus=4, local_rank=-1)

In [22]:
data_iterator = build_data_iterator(train_sum_dataset, tokenizer, batch_size=64, device=device)

In [17]:
import functools
from modeling_bertabs import BertSumOptimizer

In [18]:
model_name_or_path = "bert-base-uncased"

In [19]:
tokenizer = BertTokenizer.from_pretrained(model_name_or_path, do_lower_case=True)

I0108 21:30:22.247485 140163355588416 tokenization_utils.py:398] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/daden/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


In [20]:
from transformers import (
    AutoTokenizer,
    BertForMaskedLM,
    BertConfig,
    PreTrainedEncoderDecoder,
    Model2Model,
)
config = BertConfig.from_pretrained(model_name_or_path)

I0108 21:30:23.292389 140163355588416 configuration_utils.py:185] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /home/daden/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.bf3b9ea126d8c0001ee8a1e8b92229871d06d36d8808208cc2449280da87785c
I0108 21:30:23.293292 140163355588416 configuration_utils.py:199] Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
 

In [21]:
args= Bunch({ "max_pos": 512,
            "dec_layers": 6,
            "dec_hidden_size": 768,
            "dec_heads": 8,
            "dec_ff_size": 2048,
            "dec_dropout": 0.2,
})

In [22]:
#model = BertAbs.from_pretrained("bertabs-finetuned-cnndm")
model = BertAbs(args)

I0108 21:30:25.595635 140163355588416 configuration_utils.py:185] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /home/daden/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.bf3b9ea126d8c0001ee8a1e8b92229871d06d36d8808208cc2449280da87785c
I0108 21:30:25.596414 140163355588416 configuration_utils.py:199] Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
 

In [34]:
# Prepare the optimizer
lr = {"encoder": 0.002, "decoder": 0.2}
warmup_steps = {"encoder": 20000, "decoder": 10000}
optimizer = BertSumOptimizer(model, lr, warmup_steps)

In [29]:
def train(
    train_dataset,
    model,
    tokenizer,
    per_gpu_train_batch_size,
    n_gpus,
    device,
    max_steps,
    optimizer,
    gradient_accumulation_steps,
    num_train_epochs,
    max_grad_norm
):
    """ Fine-tune the pretrained model on the corpus. """
    # Load the data
    train_batch_size = per_gpu_train_batch_size * max(1, n_gpu)

    train_sampler = RandomSampler(train_dataset)
    model_collate_fn = functools.partial(collate, tokenizer=tokenizer, block_size=512)
    train_dataloader = DataLoader(
        train_dataset,
        sampler=train_sampler,
        batch_size=train_batch_size,
        collate_fn=model_collate_fn,
    )

    # Training schedule
    if max_steps > 0:
        num_train_epochs = max_steps // (
            len(train_dataloader) // gradient_accumulation_steps + 1
        )
    else:
        max_steps = (
            len(train_dataloader) // gradient_accumulation_steps * num_train_epochs
        )

    model.zero_grad()
    train_iterator = trange(args.num_train_epochs, desc="Epoch", disable=True)

    global_step = 0
    tr_loss = 0.0
    for _ in train_iterator:
        epoch_iterator = tqdm(train_dataloader, desc="Iteration", disable=True)
        for step, batch in enumerate(epoch_iterator):
           
            source = batch.src
            target = batch.tgt
            encoder_token_type_ids = batch.segs
            encoder_mask = batch.mask_src
            
            model.train()
            outputs = model(
                source,
                target,
                encoder_token_type_ids,
                encoder_mask,
                None,
            )

            loss = outputs[0]
            print(loss)
            if gradient_accumulation_steps > 1:
                loss /= gradient_accumulation_steps

            loss.backward()

            tr_loss += loss.item()
            if (step + 1) % gradient_accumulation_steps == 0:
                torch.nn.utils.clip_grad_norm_(model.parameters(),max_grad_norm)
                optimizer.step()
                model.zero_grad()
                global_step += 1

            if max_steps > 0 and global_step > max_steps:
                epoch_iterator.close()
                break

        if max_steps > 0 and global_step > max_steps:
            train_iterator.close()
            break

    return global_step, tr_loss / global_step

In [24]:
   train(
    train_dataset,
    model,
    tokenizer,
    per_gpu_train_batch_size,
    n_gpus,
    device,
    max_steps,
    optimizer,
    gradient_accumulation_steps,
    num_train_epochs,
    max_grad_norm
   )

  0%|          | 0/1 [00:00<?, ?it/s]

4


100%|██████████| 1/1 [00:02<00:00,  2.37s/it]


In [25]:
generated_summaries[0]

'prosecutor brice robin : " so far no videos were used in the crash investigation ". robin \'s comments follow claims by two magazines , german daily bild and french paris match. all 150 on board germanwings flight 9525 were killed'

In [26]:
data[1][0]

[' marseille prosecutor says " so far no videos were used in the crash investigation " despite media reports . <q>  journalists at bild and paris match are " very confident " the video clip is real , an editor says . <q>  andreas lubitz had informed his lufthansa training school of an episode of severe depression , airline says . <q>\n']

In [27]:
reference_summaries = []
for i in data[1]:
    reference_summaries.append(i[0].replace('<q>', ''))

In [28]:
   def _write_list_to_file(list_items, filename):
        with open(filename, "w") as filehandle:
            # for cnt, line in enumerate(filehandle):
            for item in list_items:
                filehandle.write("%s\n" % item)

In [29]:
#_write_list_to_file(generated_summaries, "./generated_summaries")

In [30]:
generated_summaries = []
with open("./generated_summaries", "r") as filehandle:
    for cnt, line in enumerate(filehandle):
        generated_summaries.append(line)

In [31]:
len(generated_summaries)

11490

In [32]:
generated_summaries[0]

'prosecutor brice robin : " so far no videos were used in the crash investigation ". robin \'s comments follow claims by two magazines , german daily bild and french paris match. all 150 on board germanwings flight 9525 were killed\n'

In [33]:
reference_summaries[0]

' marseille prosecutor says " so far no videos were used in the crash investigation " despite media reports .   journalists at bild and paris match are " very confident " the video clip is real , an editor says .   andreas lubitz had informed his lufthansa training school of an episode of severe depression , airline says . \n'

In [34]:
import rouge
import nltk

nltk.download("punkt")
rouge_evaluator = rouge.Rouge(
    metrics=["rouge-n", "rouge-l"],
    max_n=2,
    limit_length=True,
    length_limit=args.beam_size,
    length_limit_type="words",
    apply_avg=True,
    apply_best=False,
    alpha=0.5,  # Default F1_score
    weight_factor=1.2,
    stemming=True,
)

[nltk_data] Downloading package punkt to /home/daden/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [35]:
#scores = rouge_evaluator.get_scores(generated_summaries, reference_summaries)
#str_scores = format_rouge_scores(scores)

In [36]:
#scores

In [37]:
#str_scores